In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
#device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#hyper parameters
num_epocs = 15
batch_size = 4 
learning_rate = 0.004

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4
trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle=True, num_workers=2)
classes = ('plane', 'car', 'bird', ' cat','deer','dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#implement coonv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.Conv1 = nn.Conv2d(3, 40, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.Conv2 = nn.Conv2d(40, 100, 5)
        self.fc1 = nn.Linear(100*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)



    def forward(self, x):
        x = self.pool(F.relu(self.Conv1(x)))
        x = self.pool(F.relu(self.Conv2(x)))
        x = x.view(-1, 100*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
    return x

In [ ]:
model = ConvNet().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
n_total_steps = len(trainloader)
for epoch in range(num_epocs):
    for i, (images, labels) in enumerate(trainloader):
        imagess = images.to(device)
        abels = labels.to(device)
    
    #forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    #backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
        print(f'Epoch [{epoch+1}/{num_epocs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/15], Step [2000/12500], Loss: 2.3834
Epoch [1/15], Step [4000/12500], Loss: 1.6509
Epoch [1/15], Step [6000/12500], Loss: 1.1736
Epoch [1/15], Step [8000/12500], Loss: 2.7729
Epoch [1/15], Step [10000/12500], Loss: 0.7474
Epoch [1/15], Step [12000/12500], Loss: 1.5532
Epoch [2/15], Step [2000/12500], Loss: 1.3811
Epoch [2/15], Step [4000/12500], Loss: 0.7451
Epoch [2/15], Step [6000/12500], Loss: 1.1592
Epoch [2/15], Step [8000/12500], Loss: 0.6024
Epoch [2/15], Step [10000/12500], Loss: 1.3968
Epoch [2/15], Step [12000/12500], Loss: 1.2867
Epoch [3/15], Step [2000/12500], Loss: 0.7702
Epoch [3/15], Step [4000/12500], Loss: 0.2633
Epoch [3/15], Step [6000/12500], Loss: 1.3569
Epoch [3/15], Step [8000/12500], Loss: 0.2418
Epoch [3/15], Step [10000/12500], Loss: 0.7294
Epoch [3/15], Step [12000/12500], Loss: 1.0194
Epoch [4/15], Step [2000/12500], Loss: 0.4405
Epoch [4/15], Step [4000/12500], Loss: 0.5627
Epoch [4/15], Step [6000/12500], Loss: 0.4685
Epoch [4/15], Step [8000/125

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
    #max returns (value, index)

    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
        label = labels[i]
        pred = predicted[i]
        if (label == pred):
            n_class_correct[label] += 1
        n_class_samples[label] += 1
    acc = 100.0 * n_correct / n_samples
    print('accuracy of the network : {}'.format(acc))

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of {classes[i]}: {acc} %')

accuracy of the network : 71.54
accuracy of plane: 73.4 %
accuracy of car: 81.3 %
accuracy of bird: 60.6 %
accuracy of  cat: 53.7 %
accuracy of deer: 64.3 %
accuracy of dog: 71.1 %
accuracy of frog: 69.8 %
accuracy of horse: 80.8 %
accuracy of ship: 76.3 %
accuracy of truck: 84.1 %


In [ ]:
# 1st trial
# epochs = 4
# o/p - 6,16 ; 120,84
# lr - 0.001
# accuracy of the network : 44.19
# accuracy of plane: 50.3 %
# accuracy of car: 44.2 %
# accuracy of bird: 17.5 %
# accuracy of  cat: 30.6 %
# accuracy of deer: 24.9 %
# accuracy of dog: 35.9 %
# accuracy of frog: 73.8 %
# accuracy of horse: 56.0 %
# accuracy of ship: 55.1 %
# accuracy of truck: 53.6 %



# 2nd trial
# epochs = 6
# o/p - 6,16 ; 120,84
# lr - 0.001
# accuracy of the network : 49.51
# accuracy of plane: 70.3 %
# accuracy of car: 53.5 %
# accuracy of bird: 40.1 %
# accuracy of  cat: 31.6 %
# accuracy of deer: 38.8 %
# accuracy of dog: 24.3 %
# accuracy of frog: 63.1 %
# accuracy of horse: 55.5 %
# accuracy of ship: 57.7 %
# accuracy of truck: 60.2 %


# 3rd trial
# epochs = 6
# o/p - 10,25 ; 120,84
# lr - 0.001
# accuracy of the network : 54.37
# accuracy of plane: 58.1 %
# accuracy of car: 67.3 %
# accuracy of bird: 41.2 %
# accuracy of  cat: 34.1 %
# accuracy of deer: 36.1 %
# accuracy of dog: 51.4 %
# accuracy of frog: 63.9 %
# accuracy of horse: 64.0 %
# accuracy of ship: 69.0 %
# accuracy of truck: 58.6 %


# 4th trial
# epochs = 6
# o/p - 10,25 ; 120,84
# lr - 0.0009
# accuracy of the network : 52.11
# accuracy of plane: 60.1 %
# accuracy of car: 74.1 %
# accuracy of bird: 30.3 %
# accuracy of  cat: 25.6 %
# accuracy of deer: 34.2 %
# accuracy of dog: 41.7 %
# accuracy of frog: 76.3 %
# accuracy of horse: 66.9 %
# accuracy of ship: 64.4 %
# accuracy of truck: 47.5 %


# 5th trial
# epochs = 6
# o/p - 10,25 ; 120,84
# lr - 0.0015
# accuracy of the network : 57.53
# accuracy of plane: 58.3 %
# accuracy of car: 58.5 %
# accuracy of bird: 50.1 %
# accuracy of  cat: 16.2 %
# accuracy of deer: 46.1 %
# accuracy of dog: 63.2 %
# accuracy of frog: 70.3 %
# accuracy of horse: 75.2 %
# accuracy of ship: 70.2 %
# accuracy of truck: 67.2 %

# 6th trial
# epochs = 6
# o/p - 10,25 ; 120,84
# lr - 0.002
# accuracy of the network : 61.93
# accuracy of plane: 60.7 %
# accuracy of car: 66.7 %
# accuracy of bird: 50.5 %
# accuracy of  cat: 38.9 %
# accuracy of deer: 52.8 %
# accuracy of dog: 55.5 %
# accuracy of frog: 68.1 %
# accuracy of horse: 76.9 %
# accuracy of ship: 81.6 %
# accuracy of truck: 67.6 %

# 7th trial
# epochs = 6
# o/p - 10,25 ; 120,84
# lr - 0.003
# accuracy of the network : 63.95
# accuracy of plane: 76.8 %
# accuracy of car: 73.9 %
# accuracy of bird: 53.4 %
# accuracy of  cat: 45.8 %
# accuracy of deer: 50.1 %
# accuracy of dog: 46.4 %
# accuracy of frog: 72.7 %
# accuracy of horse: 73.5 %
# accuracy of ship: 78.8 %
# accuracy of truck: 68.1 %



# 8th trial
# epochs = 6
# o/p - 10,25 ; 120,84
# lr - 0.004
# accuracy of the network : 66.46
# accuracy of plane: 73.8 %
# accuracy of car: 84.8 %
# accuracy of bird: 50.4 %
# accuracy of  cat: 51.8 %
# accuracy of deer: 56.7 %
# accuracy of dog: 42.0 %
# accuracy of frog: 82.8 %
# accuracy of horse: 78.1 %
# accuracy of ship: 77.4 %
# accuracy of truck: 66.8 %


# 9th trial
# epochs = 6
# o/p - 10,25 ; 120,84
# lr - 0.005
# accuracy of the network : 65.21
# accuracy of plane: 61.1 %
# accuracy of car: 79.6 %
# accuracy of bird: 52.5 %
# accuracy of  cat: 59.2 %
# accuracy of deer: 63.4 %
# accuracy of dog: 53.6 %
# accuracy of frog: 69.7 %
# accuracy of horse: 63.0 %
# accuracy of ship: 81.8 %
# accuracy of truck: 68.2 %

# 10th trial
# epochs = 15
# o/p - 10,25 ; 120,84
# lr - 0.004
# accuracy of the network : 67.63
# accuracy of plane: 71.6 %
# accuracy of car: 86.4 %
# accuracy of bird: 53.6 %
# accuracy of  cat: 52.3 %
# accuracy of deer: 58.4 %
# accuracy of dog: 57.8 %
# accuracy of frog: 75.6 %
# accuracy of horse: 67.8 %
# accuracy of ship: 74.0 %
# accuracy of truck: 78.8 %


# 11th trial
# epochs = 15
# o/p - 20,50 ; 120,84
# lr - 0.004
# accuracy of the network : 70.22
# accuracy of plane: 78.5 %
# accuracy of car: 87.0 %
# accuracy of bird: 56.0 %
# accuracy of  cat: 52.5 %
# accuracy of deer: 58.7 %
# accuracy of dog: 55.2 %
# accuracy of frog: 79.0 %
# accuracy of horse: 78.3 %
# accuracy of ship: 78.2 %
# accuracy of truck: 78.8 %



# 12th trial
# epochs = 15
# o/p - 401,100 ; 120,84
# lr - 0.004
# accuracy of the network : 71.54
# accuracy of plane: 73.4 %
# accuracy of car: 81.3 %
# accuracy of bird: 60.6 %
# accuracy of  cat: 53.7 %
# accuracy of deer: 64.3 %
# accuracy of dog: 71.1 %
# accuracy of frog: 69.8 %
# accuracy of horse: 80.8 %
# accuracy of ship: 76.3 %
# accuracy of truck: 84.1 %
